In [22]:
import pandas as pd
import numpy as np
df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")

##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [23]:
df['gender'].value_counts(normalize=True)

Male      0.504756
Female    0.495244
Name: gender, dtype: float64

##### 2. Какое количество уникальных значений у поля InternetService?

In [24]:
len(df['InternetService'].unique())

3

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [25]:
df['TotalCharges'].replace({' ': '0'}, inplace=True)
df['TotalCharges'] = df['TotalCharges'].astype(float)
df['TotalCharges'].median(), df['TotalCharges'].mean(), df['TotalCharges'].std()

(1394.55, 2279.734303563826, 2266.7944696890195)

В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [26]:
df['PhoneService'].replace({'Yes': 1, 'No': 0}, inplace=True)

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

Все это сделал в 3 задании

##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [27]:
df['Churn'].replace({'Yes': 1, 'No': 0}, inplace=True)

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [28]:
for i in ['StreamingMovies', 'StreamingTV', 'TechSupport']:
    df[i].replace({'Yes': 1, 'No': 0, 'No internet service': 0}, inplace=True)

##### 8. Заполните пропуски в поле PhoneService значением 0

In [29]:
df['PhoneService'].fillna(0, inplace=True)

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [30]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
#Ваш код здесь

df = df[columns]
df.head(3)

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,0,29.85,0,0,0,0
1,Male,34,1,1889.50,0,0,0,0
2,Male,2,1,108.15,0,0,0,1


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [31]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'
#Ваш код здесь

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target])

##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [32]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [33]:
#Ваш код здесь
gender.fit_transform(X_train)

,gender_Female,gender_Male
4996,1,0
6056,1,0
1656,1,0
5434,0,1
6062,1,0
...,...,...
5212,0,1
5417,0,1
3154,1,0
2149,1,0


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

#### Полезно в случае использования линейных моделей

In [34]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

In [35]:
tenure.fit_transform(X_train)

array([[ 0.59169171],
       [ 1.03997013],
       [ 0.83620722],
       ...,
       [-1.16066938],
       [ 0.55093913],
       [ 1.6105063 ]])

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [36]:
#Ваш код здесь
TotalCharges = Pipeline([
    ('selector', NumberSelector('TotalCharges')),
    ('standard', StandardScaler())
])
TotalCharges.fit_transform(X_train)

array([[ 0.52661956],
       [ 0.87912401],
       [ 1.15220631],
       ...,
       [-0.86888803],
       [ 1.04385147],
       [-0.17992599]])

Объединение всех "кубиков" очень легко сделать таким образом

In [37]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [39]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [40]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('TotalCharges',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='TotalCharges')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
          

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [42]:
#Ваш код здесь
pred = pipeline.predict_proba(X_test)
pred

array([[0.94     , 0.06     ],
       [0.4122381, 0.5877619],
       [0.03     , 0.97     ],
       ...,
       [0.92     , 0.08     ],
       [0.       , 1.       ],
       [0.27     , 0.73     ]])

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [43]:
from sklearn.metrics import roc_auc_score, log_loss

#Ваш код здесь
pred = pipeline.predict(X_test)
roc_auc_score(y_test, pred), log_loss(y_test, pred)

(0.6657931068087318, 8.374896452352393)

### Сохраним наш пайплайн

In [44]:
# import dill
# with open("model_RF.dill", "wb") as f:
#     dill.dump(pipeline, f)